In [1]:
import os
import pymysql
import pandas as pd

In [7]:
con = pymysql.connect(host = "localhost"
                      , user = "spark"
                      , password = "spark"
                      , autocommit="True"
                      , charset="utf8mb4"
                      , connect_timeout = 3600
                      , cursorclass=pymysql.cursors.DictCursor
                     )
cursor = con.cursor()
cursor.execute("SELECT VERSION()")
version = cur.fetchone()
print("Database version: ", version)

Database version:  {'Database': 'casestudy'}


In [12]:
#total databases
cursor.execute("SHOW DATABASES")

6

In [13]:
list(cursor.fetchall())

[{'Database': 'casestudy'},
 {'Database': 'information_schema'},
 {'Database': 'mysql'},
 {'Database': 'performance_schema'},
 {'Database': 'retail_db'},
 {'Database': 'training'}]

In [16]:
cursor.execute("USE casestudy")

0

In [18]:
cursor.execute("SHOW tables")
cursor.fetchall()

[{'Tables_in_casestudy': 'categories'},
 {'Tables_in_casestudy': 'customers'},
 {'Tables_in_casestudy': 'departments'},
 {'Tables_in_casestudy': 'order_items'},
 {'Tables_in_casestudy': 'orders'},
 {'Tables_in_casestudy': 'products'}]

In [23]:
categories = pd.read_sql('SELECT * FROM categories', con=con)
customers = pd.read_sql('SELECT * FROM customers', con=con)
departments = pd.read_sql('SELECT * FROM departments', con=con)
order_items = pd.read_sql('SELECT * FROM order_items', con=con)
orders = pd.read_sql('SELECT * FROM orders', con=con)
products = pd.read_sql('SELECT * FROM products', con=con)

/home/rps/.local/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/rps/.local/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/rps/.local/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/rps/.local/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI

In [142]:
categories.head()

,category_id,category_department_id,category_name
0,1,2,Football
1,2,2,Soccer
2,3,2,Baseball & Softball
3,4,2,Basketball
4,5,2,Lacrosse


In [25]:
customers.head()

,customer_id,customer_fname,customer_lname,customer_email,customer_password,customer_street,customer_city,customer_state,customer_zipcode
0,1,Richard,Hernandez,XXXXXXXXX,XXXXXXXXX,6303 Heather Plaza,Brownsville,TX,78521
1,2,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126
2,3,Ann,Smith,XXXXXXXXX,XXXXXXXXX,3422 Blue Pioneer Bend,Caguas,PR,00725
3,4,Mary,Jones,XXXXXXXXX,XXXXXXXXX,8324 Little Common,San Marcos,CA,92069
4,5,Robert,Hudson,XXXXXXXXX,XXXXXXXXX,10 Crystal River Mall,Caguas,PR,00725


In [27]:
departments.head()

,department_id,department_name
0,2,Fitness
1,3,Footwear
2,4,Apparel
3,5,Golf
4,6,Outdoors


In [34]:
order_items.head()

,order_item_id,order_item_order_id,order_item_product_id,order_item_quantity,order_item_subtotal,order_item_product_price
0,1,1,957,1,299.98,299.98
1,2,2,1073,1,199.99,199.99
2,3,2,502,5,250.00,50.00
3,4,2,403,1,129.99,129.99
4,5,4,897,2,49.98,24.99


In [35]:
orders.head()

,order_id,order_date,order_customer_id,order_status
0,1,2013-07-25,11599,CLOSED
1,2,2013-07-25,256,PENDING_PAYMENT
2,3,2013-07-25,12111,COMPLETE
3,4,2013-07-25,8827,CLOSED
4,5,2013-07-25,11318,COMPLETE


In [30]:
products.head()

,product_id,product_category_id,product_name,product_description,product_price,product_image
0,1,2,Quest Q64 10 FT. x 10 FT. Slant Leg Instant U,,59.98,http://images.acmesports.sports/Quest+Q64+10+F...
1,2,2,Under Armour Men's Highlight MC Football Clea,,129.99,http://images.acmesports.sports/Under+Armour+M...
2,3,2,Under Armour Men's Renegade D Mid Football Cl,,89.99,http://images.acmesports.sports/Under+Armour+M...
3,4,2,Under Armour Men's Renegade D Mid Football Cl,,89.99,http://images.acmesports.sports/Under+Armour+M...
4,5,2,Riddell Youth Revolution Speed Custom Footbal,,199.99,http://images.acmesports.sports/Riddell+Youth+...


# Q1 
Find total sales volume ($) for all possible combinations of quarter, department and product categories. In SQL there is a technique called "rollup". If you use that, a single query can meet this requirement.


In [114]:
df=orders.copy()
df.index=df.order_date
df
df2=order_items.copy()
df.resample("Q")['order_id'].[]

order_date
2013-09-30    181729291
2013-12-31    444564751
2014-03-31    658058954
2014-06-30    830254131
2014-09-30    257861159
Freq: Q-DEC, Name: order_id, dtype: int64

In [161]:
df=order_items.groupby(['order_item_order_id'])['order_item_subtotal'].sum()
df

order_item_order_id
1         299.98
2         579.98
4         699.85
5        1129.86
7         579.92
          ...   
68879    1259.97
68880     999.77
68881     129.99
68882     109.99
68883    2149.99
Name: order_item_subtotal, Length: 57431, dtype: float64

In [156]:
orders['order_date'].min(),orders['order_date'].max()

(Timestamp('2013-07-25 00:00:00'), Timestamp('2014-07-24 00:00:00'))

In [ ]:
#sql

In [162]:
cursor.execute("""select Quarter(order_date) as Qt,order_date as date,
sum(order_item_subtotal) as total_sales from orders t1
join order_items t2
on t1.order_id=t2.order_item_order_id
group by Quarter(order_date) with rollup""")
q_sales=cursor.fetchall()
q_sales

[{'Qt': 1,
  'date': datetime.datetime(2014, 1, 3, 0, 0),
  'total_sales': 8565603.04792595},
 {'Qt': 2,
  'date': datetime.datetime(2014, 4, 3, 0, 0),
  'total_sales': 8264331.619920731},
 {'Qt': 3,
  'date': datetime.datetime(2013, 7, 25, 0, 0),
  'total_sales': 8766464.85039711},
 {'Qt': 4,
  'date': datetime.datetime(2013, 10, 4, 0, 0),
  'total_sales': 8726221.080181122},
 {'Qt': None,
  'date': datetime.datetime(2013, 10, 4, 0, 0),
  'total_sales': 34322620.59842491}]

In [169]:
cursor.execute("""select product_category_id as category_id,category_name,
sum(order_item_subtotal) as total_sales from (select category_name,
product_category_id,product_id from categories t3 join products t4 on t4.product_category_id=category_id) t1
join order_items t2
on t1.product_id=t2.order_item_product_id
group by product_category_id with rollup""")
cursor.fetchall()



[{'category_id': 2,
  'category_name': 'Soccer',
  'total_sales': 26477.049835205078},
 {'category_id': 3,
  'category_name': 'Baseball & Softball',
  'total_sales': 94057.15254592896},
 {'category_id': 4,
  'category_name': 'Basketball',
  'total_sales': 27099.329345703125},
 {'category_id': 5,
  'category_name': 'Lacrosse',
  'total_sales': 39464.78979682922},
 {'category_id': 6,
  'category_name': 'Tennis & Racquet',
  'total_sales': 44585.09062957764},
 {'category_id': 7,
  'category_name': 'Hockey',
  'total_sales': 48360.729736328125},
 {'category_id': 9,
  'category_name': 'Cardio Equipment',
  'total_sales': 3694843.196662903},
 {'category_id': 10,
  'category_name': 'Strength Training',
  'total_sales': 54895.529541015625},
 {'category_id': 11,
  'category_name': 'Fitness Accessories',
  'total_sales': 35601.440547943115},
 {'category_id': 12,
  'category_name': 'Boxing & MMA',
  'total_sales': 85205.41108703613},
 {'category_id': 13,
  'category_name': 'Electronics',
  'total

In [174]:
cursor.execute("""select t2.department_name as department_name,sum(t1.order_item_subtotal) as total_sales from 
(select order_item_subtotal,product_category_id,product_id,order_item_product_id from products t1 join
order_items t2 on t2.order_item_product_id=t1.product_id) t1 join 
(select department_name as department_name,category_id from departments t1 join 
categories t2 on t2.category_department_id=t1.department_id) 
t2 on t2.category_id=t1.product_category_id
group by department_name with rollup
""")
cursor.fetchall()

[{'department_name': 'Apparel', 'total_sales': 7323700.445373535},
 {'department_name': 'Fan Shop', 'total_sales': 17107766.279556274},
 {'department_name': 'Fitness', 'total_sales': 280044.14188957214},
 {'department_name': 'Footwear', 'total_sales': 4006498.767742157},
 {'department_name': 'Golf', 'total_sales': 4609028.242256165},
 {'department_name': 'Outdoors', 'total_sales': 995582.7216072083},
 {'department_name': None, 'total_sales': 34322620.59842491}]

In [ ]:
#category department quater combined

cursor.execute("""select Quarter(order_date),t2.department_name as department_name,category_name as category,
sum(t1.order_item_subtotal) as total_sales from (
select order_date,order_item_product_id,
order_item_subtotal from orders t1
join order_items t2
on t1.order_id=t2.order_item_order_id) t1 join 
(select t2.department_name as department_name,category_name,t1.order_item_subtotal,product_id from 
(select order_item_subtotal,product_category_id,product_id,order_item_product_id from products t1 join
order_items t2 on t2.order_item_product_id=t1.product_id) t1 join 
(select department_name as department_name,category_id,category_name from departments t1 join 
categories t2 on t2.category_department_id=t1.department_id) 
t2 on t2.category_id=t1.product_category_id)t2 on t2.product_id=t1.order_item_product_id
group by department_name,category_name,Quarter(order_date) with rollup
""")
cursor.fetchall()



# Q2
[Forecast and Account Planning] What is the quarterly moving average?

In [157]:
#sql

In [160]:
cursor.execute("""select Quarter(order_date) as Qt,avg(order_item_subtotal) as total_sales
from orders t1 join order_items t2 where t2.order_item_order_id=t1.order_id group by Quarter(order_date)""")
cursor.fetchall()

[{'Qt': 1, 'total_sales': 198.96871191465618},
 {'Qt': 2, 'total_sales': 200.34257642046813},
 {'Qt': 3, 'total_sales': 199.025242363773},
 {'Qt': 4, 'total_sales': 199.0016209847462}]

# Q3
What is the most expensive product sold by the company?

In [53]:
#pandas

In [54]:
products.head(3)

,product_id,product_category_id,product_name,product_description,product_price,product_image
0,1,2,Quest Q64 10 FT. x 10 FT. Slant Leg Instant U,,59.98,http://images.acmesports.sports/Quest+Q64+10+F...
1,2,2,Under Armour Men's Highlight MC Football Clea,,129.99,http://images.acmesports.sports/Under+Armour+M...
2,3,2,Under Armour Men's Renegade D Mid Football Cl,,89.99,http://images.acmesports.sports/Under+Armour+M...


In [92]:
df=products['product_price'].max()
df

1999.99

In [56]:
products[products["product_price"]==1999.99]

,product_id,product_category_id,product_name,product_description,product_price,product_image
207,208,10,SOLE E35 Elliptical,,1999.99,http://images.acmesports.sports/SOLE+E35+Ellip...


In [57]:
#sql

In [148]:
cursor.execute("""select max(product_price),product_name as expensive_product
from products t1 join order_items t2 
on t2.order_item_product_id=product_id""")
cursor.fetchall()

[{'max(product_price)': 1999.99,
  'expensive_product': "Diamondback Women's Serene Classic Comfort Bi"}]

# Q4
What is the average, minimum and maximum order value?

In [62]:
order_items.head()

,order_item_id,order_item_order_id,order_item_product_id,order_item_quantity,order_item_subtotal,order_item_product_price
0,1,1,957,1,299.98,299.98
1,2,2,1073,1,199.99,199.99
2,3,2,502,5,250.00,50.00
3,4,2,403,1,129.99,129.99
4,5,4,897,2,49.98,24.99


In [99]:
df=order_items['order_item_subtotal'].max()
df

1999.99

In [123]:
df=order_items.groupby(['order_item_order_id'])['order_item_subtotal'].sum()
df

order_item_order_id
1         299.98
2         579.98
4         699.85
5        1129.86
7         579.92
          ...   
68879    1259.97
68880     999.77
68881     129.99
68882     109.99
68883    2149.99
Name: order_item_subtotal, Length: 57431, dtype: float64

In [120]:
df.max()

1999.99

In [121]:
df.min()

9.99

In [122]:
df.mean()

199.3206653387379

In [149]:
orders.head(1)

,order_id,order_date,order_customer_id,order_status
0,1,2013-07-25,11599,CLOSED


In [150]:
order_items.head(1)

,order_item_id,order_item_order_id,order_item_product_id,order_item_quantity,order_item_subtotal,order_item_product_price
0,1,1,957,1,299.98,299.98


In [153]:
#sql

In [154]:
cursor.execute("""select max(order_item_subtotal) as Maximum_order,
min(order_item_subtotal) as minimum_order,avg(order_item_subtotal) as Average from order_items t1
join orders t2 on t2.order_id=t1.order_item_order_id""")
cursor.fetchall()

[{'Maximum_order': 1999.99,
  'minimum_order': 9.99,
  'Average': 199.32066922046081}]

# Q5
[Market Basket Analysis] Find frequency of each product pair. Find top 10 product pairs that are sold together. For customer XXX, which products you can recommend based on other users' purchasing patterns.


# Q6
Find the top selling product. Find whether a user XXX, has purchased that product already. This information can be used to prompt the user as a recommendation. If the customer has purchased it already, the site may ask the user whether she wants to buy it again, if not, recommend the product.

# Q7
Which customers show decline in their purchase volume over consecutive quarters. This information can be used for churn analysis.

# Q8
Which product categories and departments show decline in sales Q2Q (quarter to quarter and same quarter last year.